In [1]:
! pip install -Uq torch tensorboard sentence-transformers datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import wandb
wandb.init(mode="disabled")

#### **Create and Prepare embedding dataset**

In [3]:
from datasets import load_dataset

dataset = load_dataset("./amharic-dataset-binary", split="train")
dataset = dataset.filter(lambda row: row["label"]==1)#.select(range(10000))
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/432 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/132M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/124652 [00:00<?, ? examples/s]

Filter:   0%|          | 0/124652 [00:00<?, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'label'],
    num_rows: 31163
})

In [4]:
dataset[32]

{'query_id': 'bc76b30a40c372418a9120f7dc69c012',
 'passage_id': 'bc76b30a40c372418a9120f7dc69c012',
 'query': 'ከኢራን ጋር የተደረገው ስምምነት መተግበር ለአለም ደህንነትና ለየኢራናውያ ንየኑሮ መሻሳል እንደሚጠቅም ጆን ኬሪ አስገነዘቡ',
 'passage': '\nጆን ኬሪ ከአሜሪካ ድምጽ ሬድዮ የፋርስ ባሀርሰላጤ አገልግሎት ጋር ባደረጉት ቃለ-መጠይቅ የዩናትድ ስቴትስ የምክር ቤት አባላት በኢራንና በአሜሪካ በተመራው የስድስት ሃገሮች ቡድን መካከል የተደረገውን ስምምነት ባያጸድቅ በመካከለኛው ምስራቅ ግጭትና ጦርነት እንዲባባስ የሚያደርግ መሆኑን እንደሚረዱ እተማመናለሁ ብልዋል።“ይህን ስምምነት ባናደርግ ኑሮ የኢራኑ አያቶላ እነዚህ ሰዎች ያስገኙት ነገር ስለሌለ ከነሱ ጋር የምንደራደርበት ምክንያት የለም ባሉ ነበር። የስራ ባልደረቦቻችን በኢራን ላይ ማዕቅቦች የተጣሉት ወደ ድርድርና ስምምነት እንድታመራ ለማድረግ ነው እያሉ ማዕቀቡ እንዲቀጥል የሚደግፉበት ምክንያት ምንድነው? አሁን የተገኘውን ስምምነት ባይቀባሉ በመካከለኛው ምስራቅ ላይ ትልቅ አደጋ ይደቀናል” በማለት ጆን ኬሪ አስገንዝበዋል። የዩናትድ ስቴስ አስተዳደር ኢራን የታሰሩት አሜሪካውያን እንዲፈቱ መጠየቅን የተካተቱባቸው አጀንዳዎችን ጨምሮ መግፋት ነበረበት የሚሉ ነቀፊታዎች ሲቀርቡ ቆይተዋል። ጆን ኬሪ ታድያ ሌሎች ነጥቦችንም ጨምረን ቢሆን ኖሮ የትም አንደርስም ነበር ብለዋል።“ከዚህ በፊትም እንደገለጽነው በኢራንና በአለም ማህበረሰቡ ወይም በዩናይትድ ስቴትስ እና በኢራን መካከል በሚደረገው ድርድር ሌሎች ነጥቦችንም አክለን ቢሆን ኖሮ ለ 10 አመታት ያህል ብንደራደርም እንኳን የትም አንደርስም ነበር። ነጥቡ የኑክሌር ጉዳይ በመሆኑ ከተደረሰው መሻሻል

In [5]:
# rename columns
dataset = dataset.rename_column("query", "anchor")
dataset = dataset.rename_column("passage", "positive")

In [6]:
# Add an id column to the dataset
dataset = dataset.add_column("id", range(len(dataset)))
dataset

Flattening the indices:   0%|          | 0/31163 [00:00<?, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
    num_rows: 31163
})

In [7]:
# Split dataset into a 10% test set
dataset = dataset.train_test_split(test_size=0.1, seed=7)
dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
        num_rows: 28046
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
        num_rows: 3117
    })
})

In [ ]:
# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Creating json from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

12602505

#### **Create baseline and evaluate pretrained model**

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator, SequentialEvaluator
from sentence_transformers.util import cos_sim
from datasets import load_dataset, concatenate_datasets

# load test dataset
test_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])

corpus_dataset

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['query_id', 'passage_id', 'anchor', 'positive', 'label', 'id'],
    num_rows: 31163
})

In [ ]:
# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
) # Our corpus (cid => document)
queries = dict(
    zip(test_dataset["id"], test_dataset["anchor"])
) # Our queries (qid => question)

In [ ]:
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}
for q_id in queries:
  relevant_docs[q_id] = [q_id]

In [ ]:
matryoshka_dimensions = [512, 384, 256, 128, 64]

matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
  ir_evaluator = InformationRetrievalEvaluator(
      queries=queries,
      corpus=corpus,
      relevant_docs=relevant_docs,
      name=f"dim_{dim}",
      truncate_dim=dim,
      score_functions={"cosine": cos_sim}
  )
  matryoshka_evaluators.append(ir_evaluator)

# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

#### **Evaluate RoBERTa Medium Amharic**

In [ ]:
# BGE Small
import torch
from sentence_transformers import SentenceTransformer

model_id = "yosefw/roberta-medium-am-embed"

# Load a model
model = SentenceTransformer(
    model_id,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/169M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# Evaluate the model
results = evaluator(model)

for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_ndcg@10"
  print(f"{key}: {results[key]}")

dim_512_cosine_ndcg@10: 0.11193758612768603
dim_384_cosine_ndcg@10: 0.09887271893733758
dim_256_cosine_ndcg@10: 0.09110810850417377
dim_128_cosine_ndcg@10: 0.06627152412284452
dim_64_cosine_ndcg@10: 0.03746026207403333


In [ ]:
# print the main score
for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_recall@5"
  print(f"{key}: {results[key]}")

dim_512_cosine_recall@5: 0.13506576836701956
dim_384_cosine_recall@5: 0.11870388193776067
dim_256_cosine_recall@5: 0.10907924286172602
dim_128_cosine_recall@5: 0.0792428617260186
dim_64_cosine_recall@5: 0.04299005453962143


#### **Define loss function with Matryoshka Representation**

In [ ]:
from sentence_transformers import SentenceTransformer, SentenceTransformerModelCardData

model_id = "yosefw/roberta-medium-am-embed"

model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="RoBERTa Amharic Text Embedding Medium"
    )
)

In [ ]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [512, 384, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

#### **Fine-tune embedding model with** `SentenceTransformersTrainer`

In [ ]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")

args = SentenceTransformerTrainingArguments(
    output_dir="roberta-medium-amharic-embedding-matryoshka",
    num_train_epochs=4,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=64,
    warmup_ratio=0.1,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    fp16=True,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to=None,
    logging_steps=10,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",
)

In [ ]:
from sentence_transformers import SentenceTransformerTrainer

trainer = SentenceTransformerTrainer(
    model=model,
    args=args, # training arguments
    train_dataset=train_dataset.select_columns(
        ["anchor", "positive"]
    ), # training dataset
    loss=train_loss,
    evaluator=evaluator
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 384 Cosine Accuracy@1,Dim 384 Cosine Accuracy@3,Dim 384 Cosine Accuracy@5,Dim 384 Cosine Accuracy@10,Dim 384 Cosine Precision@1,Dim 384 Cosine Precision@3,Dim 384 Cosine Precision@5,Dim 384 Cosine Precision@10,Dim 384 Cosine Recall@1,Dim 384 Cosine Recall@3,Dim 384 Cosine Recall@5,Dim 384 Cosine Recall@10,Dim 384 Cosine Ndcg@10,Dim 384 Cosine Mrr@10,Dim 384 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
1,0.770200,No log,0.579724,0.721527,0.770292,0.822265,0.579724,0.240509,0.154058,0.082226,0.579724,0.721527,0.770292,0.822265,0.700282,0.661265,0.667261,0.579082,0.723131,0.771254,0.824190,0.579082,0.241044,0.154251,0.082419,0.579082,0.723131,0.771254,0.824190,0.701060,0.661695,0.667415,0.575553,0.709978,0.761630,0.820340,0.575553,0.236659,0.152326,0.082034,0.575553,0.709978,0.761630,0.820340,0.695694,0.656120,0.661918,0.547000,0.700353,0.747193,0.803337,0.547000,0.233451,0.149439,0.080334,0.547000,0.700353,0.747193,0.803337,0.674556,0.633393,0.639452,0.510427,0.661854,0.712544,0.777671,0.510427,0.220618,0.142509,0.077767,0.510427,0.661854,0.712544,0.777671,0.641977,0.598841,0.605080,0.641977
2,0.223200,No log,0.608598,0.750080,0.798203,0.852743,0.608598,0.250027,0.159641,0.085274,0.608598,0.750080,0.798203,0.852743,0.729884,0.690680,0.695582,0.610523,0.751363,0.798524,0.849214,0.610523,0.250454,0.159705,0.084921,0.610523,0.751363,0.798524,0.849214,0.729334,0.690980,0.696187,0.602502,0.742060,0.792429,0.849856,0.602502,0.247353,0.158486,0.084986,0.602502,0.742060,0.792429,0.849856,0.724954,0.685186,0.690122,0.585178,0.724415,0.774783,0.834777,0.585178,0.241472,0.154957,0.083478,0.585178,0.724415,0.774783,0.834777,0.707030,0.666505,0.671920,0.556304,0.706769,0.755213,0.811036,0.556304,0.235590,0.151043,0.081104,0.556304,0.706769,0.755213,0.811036,0.682351,0.641321,0.647134,0.682351
3,0.057200,No log,0.629451,0.762913,0.805582,0.861084,0.629451,0.254304,0.161116,0.086108,0.629451,0.762913,0.805582,0.861084,0.744055,0.706824,0.711535,0.625602,0.765159,0.805582,0.859480,0.625602,0.255053,0.161116,0.085948,0.625602,0.765159,0.805582,0.859480,0.742209,0.704803,0.709678,0.623356,0.757780,0.798524,0.853064,0.623356,0.252593,0.159705,0.085306,0.623356,0.757780,0.798524,0.853064,0.737580,0.700748,0.705939,0.604748,0.740776,0.788258,0.845685,0.604748,0.246925,0.157652,0.084568,0.604748,0.740776,0.788258,0.845685,0.723456,0.684633,0.689894,0.578441,0.722169,0.770292,0.823227,0.578441,0.240723,0.154058,0.082323,0.578441,0.722169,0.770292,0.8232

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=876, training_loss=0.6072702435734065, metrics={'train_runtime': 2207.959, 'train_samples_per_second': 50.809, 'train_steps_per_second': 0.397, 'total_flos': 0.0, 'train_loss': 0.6072702435734065, 'epoch': 3.9840546697038723})

In [ ]:
# save the best model
trainer.save_model()

#### **Evaluate fine-tuned model against baseline**

In [ ]:
from sentence_transformers import SentenceTransformer

fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)

# Evaluate the model
results = evaluator(fine_tuned_model)

# print the main score
for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_ndcg@10"
  print(f"{key}: {results[key]}")

dim_512_cosine_ndcg@10: 0.7440606561907367
dim_384_cosine_ndcg@10: 0.7424192541161443
dim_256_cosine_ndcg@10: 0.7378921249266414
dim_128_cosine_ndcg@10: 0.7233474573102769
dim_64_cosine_ndcg@10: 0.700565832342667


In [ ]:
# print the main score
for dim in matryoshka_dimensions:
  key = f"dim_{dim}_cosine_recall@5"
  print(f"{key}: {results[key]}")

dim_512_cosine_recall@5: 0.8055822906641001
dim_384_cosine_recall@5: 0.8055822906641001
dim_256_cosine_recall@5: 0.7988450433108758
dim_128_cosine_recall@5: 0.7879371190247032
dim_64_cosine_recall@5: 0.7706127686878409


In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 510, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 512, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# The sentences to encode
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "በአክሱም ከተማ የሚገኙ ሙስሊም ሴት ተማሪዎች ከሒጃብ መልበስ ጋር በተያያዘ ውዝግብ ከትምህርት ገበታ ውጭ ሆነው እንደሚገኙ የትግራይ እስልምና ጉዳዮች ምክርቤት ስታወቀ። ይህን ለመፍታት ከክልሉ ትምህርት ቢሮ ጋር ንግግር ላይ መሆኑም የክልሉ እስልምና ጉዳዮች ምክርቤት ለዶቼቬለ ገልጿል።",
  "በማዕከላዊ ኢትዮጵያ ክልል ሃድያ ዞን ጊቤ ወረዳ በሚገኙ 12 ቀበሌዎች መሠረታዊ የመንግሥት አገልግሎት መስጫ ተቋማት በሙሉና በከፊል በመዘጋታቸው መቸገራቸውን ነዋሪዎች አመለከቱ። ከባለፈው ዓመት ጀምሮ የጤና፣ የትምህርት እና የግብር አሰባሰብ ሥራዎች በየአካባቢያቸው እየተከናወኑ አለመሆናቸውንም ለዶቼ ቬለ ተናግረዋል።",
  "የሕዝብ ተወካዮች ምክር ቤት አባል እና የቋሚ ኮሚቴ ሰብሳቢ የነበሩት አቶ ክርስቲያን ታደለ እና የአማራ ክልል ምክር ቤት አባል የሆኑት አቶ ዮሐንስ ቧያለው ከቃሊቲ ወደ ቂሊንጦ ማረሚያ ቤት መዛወራቸውን ጠበቃቸው ተናገሩ።",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡"
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)

(5, 512)
tensor([[ 1.0000,  0.1265,  0.2156, -0.0826,  0.7958],
        [ 0.1265,  1.0000,  0.2543,  0.3181,  0.2156],
        [ 0.2156,  0.2543,  1.0000,  0.2141,  0.3645],
        [-0.0826,  0.3181,  0.2141,  1.0000,  0.0859],
        [ 0.7958,  0.2156,  0.3645,  0.0859,  1.0000]])


In [ ]:
from google.colab import userdata

# push model to hub
# trainer.model.push_to_hub("roberta-amharic-text-embedding-medium", token=userdata.get("HF_WRITE"))

model.safetensors:   0%|          | 0.00/169M [00:00<?, ?B/s]

'https://huggingface.co/rasyosef/roberta-amharic-text-embedding-medium/commit/b92e808507b854a4e749ae8d3222d13dba55e0c3'

In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("rasyosef/roberta-amharic-text-embedding-medium")
# Run inference
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡",
  "ለኢትዮጵያ ብሔራዊ ባንክ ዋጋን የማረጋጋት ቀዳሚ ዓላማ ጋር የተጣጣሙ የገንዘብ ፖሊሲ ምክረ ሀሳቦችን እንዲሰጥ የተቋቋመው የኢትዮጵያ ብሔራዊ ባንክ የገንዘብ ፖሊሲ ኮሚቴ እስካለፈው ህዳር ወር የነበረው እአአ የ2024 የዋጋ ግሽበት በተለይምምግብ ነክ ምርቶች ላይ ከአንድ ዓመት በፊት ከነበው ጋር ሲነጻጸር መረጋጋት ማሳየቱን ጠቁሟል፡፡ ዶይቼ ቬለ ያነጋገራቸው የአዲስ አበባ ነዋሪዎች ግን በዚህ የሚስማሙ አይመስልም፡፡ ከአምና አንጻር ያልጨመረ ነገር የለም ባይ ናቸው፡፡ የኢኮኖሚ  ባለሙያም በሰጡን አስተያየት ጭማሪው በሁሉም ረገድ የተስተዋለ በመሆኑ የመንግስት ወጪን በመቀነስ ግብርናው ላይ አተኩሮ መስራት ምናልባትም የዋጋ መረጋጋቱን ሊያመጣ ይችላል ይላሉ፡፡"
]
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 512]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities, similarities.shape)
# [3, 3]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/673 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

(3, 512)
tensor([[ 1.0000,  0.6473,  0.0116],
        [ 0.6473,  1.0000, -0.0354],
        [ 0.0116, -0.0354,  1.0000]]) torch.Size([3, 3])


In [ ]:
sentences = [
  "የተደጋገመው የመሬት መንቀጥቀጥና የእሳተ ገሞራ ምልክት በአፋር ክልል",
  "የዋጋ ግሽበት በባለሙያዎች እና ነዋሪዎች አተያይ",
  "ከተደጋጋሚ መሬት መንቀጥቀጥ በኋላ አፋር ክልል እሳት ከመሬት ውስጥ ሲፈላ ታይቷል፡፡ ከመሬት ውስጥ እሳትና ጭስ የሚተፋው እንፋሎቱ ዛሬ ማለዳውን 11 ሰዓት ግድም ከከባድ ፍንዳታ በኋላየተስተዋለ መሆኑን የአከባቢው ነዋሪዎች እና ባለስልጣናት ለዶቼ ቬለ ተናግረዋል፡፡ አለት የሚያፈናጥር እሳት ነው የተባለው እንፋሎቱ በክልሉ ጋቢረሱ (ዞን 03) ዱለቻ ወረዳ ሰጋንቶ ቀበሌ መከሰቱን የገለጹት የአከባቢው የአይን እማኞች ከዋናው ፍንዳታ በተጨማሪ በዙሪያው ተጨማሪ ፍንዳታዎች መታየት ቀጥሏል ባይ ናቸው፡፡",
  "ለኢትዮጵያ ብሔራዊ ባንክ ዋጋን የማረጋጋት ቀዳሚ ዓላማ ጋር የተጣጣሙ የገንዘብ ፖሊሲ ምክረ ሀሳቦችን እንዲሰጥ የተቋቋመው የኢትዮጵያ ብሔራዊ ባንክ የገንዘብ ፖሊሲ ኮሚቴ እስካለፈው ህዳር ወር የነበረው እአአ የ2024 የዋጋ ግሽበት በተለይምምግብ ነክ ምርቶች ላይ ከአንድ ዓመት በፊት ከነበው ጋር ሲነጻጸር መረጋጋት ማሳየቱን ጠቁሟል፡፡ ዶይቼ ቬለ ያነጋገራቸው የአዲስ አበባ ነዋሪዎች ግን በዚህ የሚስማሙ አይመስልም፡፡ ከአምና አንጻር ያልጨመረ ነገር የለም ባይ ናቸው፡፡ የኢኮኖሚ  ባለሙያም በሰጡን አስተያየት ጭማሪው በሁሉም ረገድ የተስተዋለ በመሆኑ የመንግስት ወጪን በመቀነስ ግብርናው ላይ አተኩሮ መስራት ምናልባትም የዋጋ መረጋጋቱን ሊያመጣ ይችላል ይላሉ፡፡"
]
embeddings = model.encode(sentences)

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[ 1.0000,  0.1729,  0.6473,  0.0116],
        [ 0.1729,  1.0000,  0.2025,  0.5641],
        [ 0.6473,  0.2025,  1.0000, -0.0354],
        [ 0.0116,  0.5641, -0.0354,  1.0000]])


In [ ]:
sentences = [
  "አበበ በሶ በላ",
  "በቀለ ውሃ ጠጣ",
  "የኦፓል ማዕድን አውጪ ወጣቶች ስሞታ"
]
embeddings = model.encode(sentences)

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[1.0000, 0.4743, 0.3108],
        [0.4743, 1.0000, 0.3775],
        [0.3108, 0.3775, 1.0000]])
